In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [8]:
cfg_file = "configs/kpconv_semantic3d_OPTIMIZEDFORTRAINING.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = "/mnt/d/Krishna/Research/Datasets/Semantic3D/"
dataset = ml3d.datasets.Semantic3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# download the weights.
ckpt_folder = "logs/KPFCNN_Semantic3D_torch/checkpoint/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "ckpt_08800.pth"

In [9]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

In [10]:
dataset.get_split_list("test")

array(['/mnt/d/Krishna/Research/Datasets/Semantic3D/IIIT_Base.txt'],
      dtype='<U57')

In [11]:
test_split = dataset.get_split("test")
data = test_split.get_data(0)

In [12]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

test 0/1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1417791/1417791 [44:50<00:00, 345.29it/s]

In [15]:
len(result['predict_labels'])

2275320

In [16]:
set(result['predict_labels'])

{0, 2, 4}

In [17]:
newData = data

In [18]:
newData['label'] = result['predict_labels']

In [19]:
newData

{'point': array([[-10.417     ,  -0.18200001,  -0.7285    ],
        [-10.302     ,  -0.23      ,  -0.779     ],
        [-10.325     ,  -0.23475   ,  -0.71099997],
        ...,
        [ 13.417     ,  10.72      ,  -0.692     ],
        [ 13.421     ,  10.711     ,  -0.685     ],
        [ 13.512     ,  11.25      ,  -0.745     ]], dtype=float32),
 'feat': array([[  0.      ,   0.      ,   0.483051],
        [  0.      ,   0.      ,   0.432203],
        [  0.      ,   0.      ,   0.483051],
        ...,
        [148.      , 148.      ,   0.584746],
        [151.      , 151.      ,   0.597458],
        [157.      , 157.      ,   0.618644]], dtype=float32),
 'intensity': array([  0.,   0.,   0., ..., 148., 151., 157.], dtype=float32),
 'label': array([0, 0, 0, ..., 0, 0, 0])}

In [20]:
# evaluate performance on the test set; this will write logs to './logs'.
#pipeline.run_test()

In [21]:
import numpy as np

# Assuming newData is already defined as in your example

# Combine all data into a single array for writing to a text file
# Structure: x, y, z, r, g, b, label
data_to_write = np.hstack((
    newData['point'],                       # x, y, z coordinates
    newData['feat'][:, :3],                 # r, g, b colors
    newData['label'].reshape(-1, 1)         # label as a single column
))

# Save to a text file ckpt_00300
np.savetxt("output_point_cloud_08800.txt", data_to_write, fmt='%.6f %.6f %.6f %.3f %.3f %.3f %d')

print("Point cloud data with labels saved to output_point_cloud.txt")

Point cloud data with labels saved to output_point_cloud.txt


In [ ]:
import numpy as np

# Assuming newData is already defined as in your example

# Combine the point cloud data: x, y, z, r, g, b, label
# Ensure colors are normalized to [0, 1] range
colors = newData['feat'][:, :3] / 255.0
labels = newData['label'].reshape(-1, 1).astype(np.float32)
data_to_write = np.hstack((newData['point'], colors, labels))

# Create a header for the PCD file
header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z rgb label
SIZE 4 4 4 4 4
TYPE F F F F F
COUNT 1 1 1 1 1
WIDTH {data_to_write.shape[0]}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {data_to_write.shape[0]}
DATA ascii
"""

# Convert RGB values from float (0-1) to packed float (for PCD)
rgb_packed = np.array(
    ((colors[:, 0] * 255).astype(np.uint32) << 16) |
    ((colors[:, 1] * 255).astype(np.uint32) << 8) |
    ((colors[:, 2] * 255).astype(np.uint32))
).view(np.float32).reshape(-1, 1)

# Combine points, packed RGB, and labels
pcd_data = np.hstack((newData['point'], rgb_packed, labels))

# Write to the PCD file
with open("output_point_cloud_06500", "w") as f:
    f.write(header)
    np.savetxt(f, pcd_data, fmt="%.6f %.6f %.6f %.6f %.1f")

print("Point cloud with labels saved to output_with_labels.pcd")


test 0/1: 100%|███████████████████████████████████████████████████████████████| 692214/692214 [00:54<00:00, 2143.62it/s]